In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

Dataset: CIFAR 10, resize image to 224x224

In [31]:
cifar = torchvision.datasets.CIFAR10(root="../data/", train=True, 
                                     transform=None, target_transform=None, 
                                     download=True)
batch_size = 128
train_loader = torch.utils.data.DataLoader(
    dataset=cifar,
    train=True,
    batch_size=128,
    shuffle=True
)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

0.2%

KeyboardInterrupt: 

Hãy xem thử dữ liệu có dạng như thế nào?

Define the Network architechture

<figure>
    <img src="./img/AlexNetArchitecture.png" width="25%" class="center"/>
    <figcaption>Cấu trúc mạng AlexNet</figcaption>
</figure>

In [19]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__() # Gọi hàm __init__() của `nn.Module`
        
        # Sau đó định nghĩa các tầng Conv2d trong mạng tại đây luôn.
        # Khối MaxPool không cần định nghĩa, ta thực hiện phép toán đó tại phương thức `forward`.
        
        # Tầng đầu tiên trên hình, 11x11 Conv2d 
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4)
        # Tương tự với các tầng Conv2d tiếp theo, số kênh đầu ra của tầng trước là số kênh đầu vào
        # của tầng hiện tại:
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=2)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, padding=1)
        
        # Đến 3 tầng kết nối đầy đủ (Fully-connected Layer, hay Dense). 
        # Xác định đối số `in_features` cho tầng thứ nhất bằng tích số kênh tầng Conv2d cuối (conv5) 
        # với kích thước ảnh sau tầng đó.
        
        self.fc1 = nn.Linear(in_features=384*2*2, out_features=4096) # Chỗ này có cách lấy `in_features` mà ko cần tính toán nhức đầu
        self.fc2 = nn.Linear(in_features=4096, out_features=4096)
        
        # Tầng FC cuối cùng, đối số out_features ở đây bằng số lượng nhãn trong MNIST dataset, là 10, 
        # không phải 1000 như hình trên (hình trong paper gốc).
        self.fc3 = nn.Linear(in_features=4096, out_features=10)

    def forward(self, X):
        # Hàm này định nghĩa cách ảnh đầu vào X (dạng tensor) được truyền qua từng tầng của mạng.
        X = X.type(torch.float) # chuyển kiểu từ int sang float

        X = F.relu(self.conv1(X))
        X = torch.max_pool2d(X, kernel_size=3, stride=2)
        X = F.relu(self.conv2(X))
        X = torch.max_pool2d(X, kernel_size=3, stride=2)
        X = F.relu(self.conv3(X))
        X = F.relu(self.conv4(X))
        X = F.relu(self.conv5(X))
        X = F.max_pool2d(X, kernel_size=3, stride=2)
#         print(X.size()) # In dòng này để tính shape cho tầng fc1.
        
        X = torch.flatten(X, start_dim=1) # Làm phẳng tensor 4D thành tensor 2D để đưa vào tầng FC.
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        # Sử dụng hàm kích hoạt `softmax` ở tầng cuối cùng để tính xác suất 10 nhãn đầu ra.
        X = F.softmax(self.fc3(X))
        return X

In [ ]:
# X = torch.randn((1,3,224,224))
# y=AlexNet()(X)

In [22]:
net = AlexNet()

In [25]:
# Thử in ra kích thước các tham số của mạng
for i, p in enumerate(net.parameters()):
    print(i, p.size())

0 torch.Size([96, 3, 11, 11])
1 torch.Size([96])
2 torch.Size([256, 96, 5, 5])
3 torch.Size([256])
4 torch.Size([384, 256, 3, 3])
5 torch.Size([384])
6 torch.Size([384, 384, 3, 3])
7 torch.Size([384])
8 torch.Size([384, 384, 3, 3])
9 torch.Size([384])
10 torch.Size([4096, 384])
11 torch.Size([4096])
12 torch.Size([4096, 4096])
13 torch.Size([4096])
14 torch.Size([10, 4096])
15 torch.Size([10])


Define the loss function and optimizer

In [28]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),lr=0.01)

Training

In [2]:
n_epochs = 10
losses = [] # Lưu các loss sau mỗi 100 batch để vẽ learning curve

for epoch in range(n_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader): # Với mỗi batch:
        X, y = data # data là 1 tuple (X, y), dòng này unpack tuple đó để gán vào X, y.
        optimizer.zero_grad() # Reset các gradient trọng số sau mỗi lần cập nhật trọng số ở mỗi batch, nếu không chúng sẽ tích luỹ, gây ảnh hưởng đến các batch sau.
        
        predictions = net(X) # Thực hiện lượt truyền xuôi (forward pass) cho batch hiện tại -> có kết quả nhãn dự đoán 
        loss = criterion(predictions, y) # Tính hàm mất mát dựa vào dự đoán và nhãn gốc
        loss.backward() # Thực hiện lan truyền ngược (backward pass)
        
        optimizer.step() # Cập nhật các tham số của mô hình
        
        running_loss += loss.item()
        if epoch%100 == 99:
            avg_loss = running_loss/100.0
            losses.append(avg_loss)
            running_loss = 0.0
            print("Epoch {}, minibatch {}: loss = {}".format(epoch+1, i+1, avg_loss))
        

NameError: name 'train_loader' is not defined